In [1]:
import pysam

In [ ]:
variant = 'chr6_31268790_T_C'
sample = 'YRI2'
bam = '/Users/fairliereese/Documents/programming/mele_lab/projects/240903_pt/data/03_mapping/pantrx_general_mapping/genomic/7_NI2_GM18489.bam'
